## CMPE 258 - HW 5
## Abhishek Bais

## This collab is modified version of this [notebook](https://github.com/sk48880/Kaggle-New-York-Taxi-Fare-Prediction/blob/master/NYC%20Taxi%20Fare%20Prediction%20Complete%20EDA.ipynb) by Sandeep Kumar.

### Importing Data and Libraries

In [1]:
# load some default Python modules
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

# Data injestion

## 1.0. Abhishek Added -  Upload Data for Chicago Taxi

In [2]:
!wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv

--2021-04-18 22:25:22--  https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1922812 (1.8M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.83M  --.-KB/s    in 0.1s    

2021-04-18 22:25:23 (17.3 MB/s) - ‘data.csv’ saved [1922812/1922812]



## 2.0. Abhishek Added - Data injestion

In [141]:
df = pd.read_csv('/content/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15002 entries, 0 to 15001
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   pickup_community_area   15000 non-null  float64
 1   fare                    15002 non-null  float64
 2   trip_start_month        15002 non-null  int64  
 3   trip_start_hour         15002 non-null  int64  
 4   trip_start_day          15002 non-null  int64  
 5   trip_start_timestamp    15002 non-null  int64  
 6   pickup_latitude         15000 non-null  float64
 7   pickup_longitude        15000 non-null  float64
 8   dropoff_latitude        14519 non-null  float64
 9   dropoff_longitude       14519 non-null  float64
 10  trip_miles              15002 non-null  float64
 11  pickup_census_tract     1 non-null      float64
 12  dropoff_census_tract    10761 non-null  float64
 13  payment_type            15002 non-null  object 
 14  company                 9862 non-null 

## 3.0. Abhishek Added - Data preprocessing, drop useless columns

In [142]:
##Drop useless columns
df = df.drop(['trip_start_timestamp','trip_miles','pickup_census_tract',
              'dropoff_census_tract','trip_seconds','payment_type','tips', 
              'company','dropoff_community_area','pickup_community_area'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15002 entries, 0 to 15001
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fare               15002 non-null  float64
 1   trip_start_month   15002 non-null  int64  
 2   trip_start_hour    15002 non-null  int64  
 3   trip_start_day     15002 non-null  int64  
 4   pickup_latitude    15000 non-null  float64
 5   pickup_longitude   15000 non-null  float64
 6   dropoff_latitude   14519 non-null  float64
 7   dropoff_longitude  14519 non-null  float64
dtypes: float64(5), int64(3)
memory usage: 937.8 KB


## 4.0. Abhishek Added - Data preprocessing, drop fares > $200

In [143]:
#Drop NA rows
df = df.dropna()

#Drop rows where FARE > 200( only 1 row found )
df = df.drop(df[df['fare'] > 200].index)

#Change datatype of cols
df['trip_start_month'] = df['trip_start_month'].astype('str')
df['trip_start_hour'] = df['trip_start_hour'].astype('str')
df['trip_start_day'] = df['trip_start_day'].astype('str')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14518 entries, 28 to 15000
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fare               14518 non-null  float64
 1   trip_start_month   14518 non-null  object 
 2   trip_start_hour    14518 non-null  object 
 3   trip_start_day     14518 non-null  object 
 4   pickup_latitude    14518 non-null  float64
 5   pickup_longitude   14518 non-null  float64
 6   dropoff_latitude   14518 non-null  float64
 7   dropoff_longitude  14518 non-null  float64
dtypes: float64(5), object(3)
memory usage: 1020.8+ KB


## 5.0. Abhishek Added - Data preprocessing, inspection of train/ test

In [144]:
##Train test split ( Original DF )
np.random.seed(seed=2)
msk = np.random.rand(len(df)) < 0.85
df_train = df[msk]
df_test = df[~msk]

print(len(df_train))
print(len(df_test))

df_train.to_csv("train.csv", index=False, header=False)
df_test.to_csv("eval.csv", index=False, header=False)

12347
2171


## 6.0. Abhishek Added - Data preprocessing, inspect fares > 0

In [145]:
# Inspect fares > 0
df_train = df_train.loc[df_train.fare > 0]
df_train.fare.min()

0.01

In [146]:
len(df_train)

12334

In [147]:
train_X = df_train.drop(columns=['fare'])
Y = np.log(df_train.fare)
test_X = df_test

In [148]:
df = train_X.append(test_X,sort=False)

In [149]:
# Basic Stats of the data set
df_train.describe()

,fare,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
count,12334.000000,12334.000000,12334.000000,12334.000000,12334.000000
mean,11.264938,41.902461,-87.654612,41.902655,-87.653982
std,9.316211,0.036713,0.062996,0.038628,0.056417
min,0.010000,41.694879,-87.913625,41.663671,-87.913625
25%,5.850000,41.880994,-87.655879,41.880994,-87.656804
50%,7.850000,41.892508,-87.632746,41.893216,-87.634156
75%,12.050000,41.916356,-87.626211,41.922686,-87.626215
max,112.650000,42.009623,-87.572782,42.021224,-87.540936


In [150]:
train_X = df_train.drop(columns=['fare'])
Y = np.log(df_train.fare)
test_X = df_test

In [173]:
df_test.columns

Index(['pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
       'dropoff_longitude', 'dist_km'],
      dtype='object')

## 7.0. Abhishek added - Function to compute distance

In [151]:
def haversine_distance(df, lat1, long1, lat2, long2):
    """
    Calculates the haversine distance between 2 sets of GPS coordinates in df
    """
    r = 6371  # average radius of Earth in kilometers
       
    phi1 = np.radians(df[lat1])
    phi2 = np.radians(df[lat2])
    
    delta_phi = np.radians(df[lat2]-df[lat1])
    delta_lambda = np.radians(df[long2]-df[long1])
     
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = (r * c) # in kilometers

    return d

In [152]:
df['dist_km'] = haversine_distance(df,'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude')
df.head()

,trip_start_month,trip_start_hour,trip_start_day,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,fare,dist_km
28,10,1,1,41.952823,-87.653244,41.920452,-87.679955,NaN,4.223525
29,3,15,6,41.899670,-87.669838,41.920452,-87.679955,NaN,2.457759
30,8,23,5,41.906026,-87.675312,41.920452,-87.679955,NaN,1.649418
31,10,21,7,41.906651,-87.665338,41.920452,-87.679955,NaN,1.953926
32,11,11,2,41.871016,-87.631407,41.849247,-87.624135,NaN,2.494392


## 8.0. Abhishek added - Prepare the model

In [153]:
cont_cols = ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'passenger_count', 'dist_km']


## 9.0. Abhishek added - Keep relevant columns

In [154]:
# keep only the cols for the model
df = df[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'dist_km']]

## 10.0. Abhishek Added Prepare the df_train, df_test

In [155]:
df_train = df[:12334]
df_test = df[12334:]

In [156]:
#Normalizing 

from pandas.api.types import is_numeric_dtype

#"Compute the means and stds of `self.cont_names` columns to normalize them."
def Normalize(df):
  means,stds = {},{}
  cont_names = ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']
  for n in cont_names:
    assert is_numeric_dtype(df[n]), (f"""Cannot normalize '{n}' column as it isn't numerical. Are you sure it doesn't belong in the categorical set of columns?""")
    means[n],stds[n] = df[n].mean(),df[n].std()
    df[n] = (df[n]-means[n]) / (1e-7 + stds[n])


Normalize(df_train)
Normalize(df_test)
X = df_train

In [157]:
X.shape

(12334, 5)

In [158]:
Y.shape

(12334,)

In [159]:
from sklearn.model_selection import train_test_split 
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.20, random_state=142,shuffle=True ) 
X_train.head()

,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,dist_km
9032,-0.314306,0.567394,-0.560755,0.376407,1.593253
10785,-0.584714,0.347093,-0.111930,0.588817,2.168064
7718,-0.077877,0.338179,1.168477,-0.529139,6.796111
1292,-0.642701,0.467010,-0.079044,0.366453,2.401673
1337,-0.642701,0.467010,-0.741688,-0.169012,3.218814


## Modelling

In [160]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

### Linear Baseline

## 11.0. Abhishek updated K-Folded cross validation to 6 fold

In [163]:
#Validation function
n_folds = 6

def rmse_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train, 
                                   y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [164]:
linear = make_pipeline(RobustScaler(), LinearRegression())
score = rmse_cv(linear)
print("\nLinear score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Linear score: 0.3236 (0.0214)



In [167]:
linear.fit(X_train,y_train)
y_lin_pred = linear.predict(X_val)
print("test score", rmse(y_val,y_lin_pred))
prediction = linear.predict(df_test).tolist()


test score 0.34270642083124553


In [168]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
score = rmse_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 0.3236 (0.0214)



In [169]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
score = rmse_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ElasticNet score: 0.3236 (0.0214)



### Baseline XGB

In [170]:
params = {
   
    'max_depth': 7,
    'gamma' :0,
    'eta':.03, 
    'subsample': 1,
    'colsample_bytree': 0.9, 
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'silent': 0
}

In [174]:
def XGBmodel(X_train,X_test,y_train,y_test,params):
    matrix_train = xgb.DMatrix(X_train,label=y_train)
    matrix_test = xgb.DMatrix(X_test,label=y_test)
    model=xgb.train(params=params,
                    dtrain=matrix_train,num_boost_round=5000, 
                    early_stopping_rounds=10,evals=[(matrix_test,'test')])
    return model

model = XGBmodel(X_train,X_val,y_train,y_val,params)

[23:08:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	test-rmse:1.74923
Will train until test-rmse hasn't improved in 10 rounds.
[1]	test-rmse:1.69809
[2]	test-rmse:1.64941
[3]	test-rmse:1.60135
[4]	test-rmse:1.55476
[5]	test-rmse:1.50964
[6]	test-rmse:1.46589
[7]	test-rmse:1.42353
[8]	test-rmse:1.38252
[9]	test-rmse:1.34355
[10]	test-rmse:1.30505
[11]	test-rmse:1.26854
[12]	test-rmse:1.23238
[13]	test-rmse:1.19736
[14]	test-rmse:1.1635
[15]	test-rmse:1.13068
[16]	test-rmse:1.09961
[17]	test-rmse:1.06888
[18]	test-rmse:1.03915
[19]	test-rmse:1.01038
[20]	test-rmse:0.983149
[21]	test-rmse:0.956206
[22]	test-rmse:0.930722
[23]	test-rmse:0.906108
[24]	test-rmse:0.88174
[25]	test-rmse:0.858169
[26]	test-rmse:0.835403
[27]	test-rmse:0.813393
[28]	test-rmse:0.79214
[29]	test-rmse:0.771631
[30]	test-rmse:0.752382
[31]	test-rmse:0.733225
[32]	test-rmse:0.714761
[33]	test-rmse:0.696959
[34]	test-rmse:0.679788
[35]	test

In [175]:
prediction = model.predict(xgb.DMatrix(df_test), ntree_limit = model.best_ntree_limit).tolist()


#### Baseline LGBM

In [176]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

In [177]:
# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'max_depth': 7,
    'verbose': 0
}

## 12.0 Abhishek added hyper-param tuning

In [186]:
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=999,
                valid_sets=lgb_eval,
                early_stopping_rounds=20)

[1]	valid_0's rmse: 0.579144
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's rmse: 0.536777
[3]	valid_0's rmse: 0.500029
[4]	valid_0's rmse: 0.468029
[5]	valid_0's rmse: 0.440483
[6]	valid_0's rmse: 0.416927
[7]	valid_0's rmse: 0.396833
[8]	valid_0's rmse: 0.37983
[9]	valid_0's rmse: 0.36543
[10]	valid_0's rmse: 0.353495
[11]	valid_0's rmse: 0.343485
[12]	valid_0's rmse: 0.335165
[13]	valid_0's rmse: 0.328301
[14]	valid_0's rmse: 0.32258
[15]	valid_0's rmse: 0.318147
[16]	valid_0's rmse: 0.314447
[17]	valid_0's rmse: 0.311375
[18]	valid_0's rmse: 0.309073
[19]	valid_0's rmse: 0.306952
[20]	valid_0's rmse: 0.305226
[21]	valid_0's rmse: 0.30402
[22]	valid_0's rmse: 0.302827
[23]	valid_0's rmse: 0.302035
[24]	valid_0's rmse: 0.301334
[25]	valid_0's rmse: 0.30081
[26]	valid_0's rmse: 0.30031
[27]	valid_0's rmse: 0.300127
[28]	valid_0's rmse: 0.299859
[29]	valid_0's rmse: 0.29956
[30]	valid_0's rmse: 0.299357
[31]	valid_0's rmse: 0.299249
[32]	valid_0's rmse: 0.2

In [187]:
prediction = gbm.predict(df_test,num_iteration= gbm.best_iteration).tolist()


In [188]:
prediction

[1.4572878309067812,
 2.1159545358596876,
 1.743365048421859,
 2.2766739139234695,
 2.2766739139234695,
 1.863297789480417,
 2.3829220496593453,
 2.973907233726699,
 3.7594515627535836,
 2.0758102647714187,
 1.6150132308745986,
 2.038009709635761,
 2.751187935771213,
 1.9716584449363521,
 1.9716584449363521,
 2.078943126352108,
 2.5242042074505138,
 1.6094300010911158,
 1.6549452859584888,
 1.9119563916870526,
 2.276410309983857,
 1.7164971866701957,
 1.781907082355457,
 1.9339632465137555,
 2.28254002274039,
 1.7841298380527255,
 2.2415795921044195,
 2.9833652263069994,
 1.6372939687852166,
 1.7775005565376998,
 1.978463858148348,
 2.7085851541099233,
 2.5223506738721504,
 2.4149343262372147,
 2.268723216812954,
 1.7546002727875416,
 2.649835895172459,
 1.5711074519214776,
 2.1890726642904377,
 2.5579190592741945,
 2.164114424712085,
 2.6953597322510534,
 2.1199883305045835,
 2.7883519158271435,
 2.3545188688653385,
 1.60908927514831,
 2.6527648962556465,
 2.121552466368692,
 2.060740

### Ensemble -- Weighted Average of XGB and LGB

In [189]:
lgb_baseline = gbm.predict(df_test,num_iteration= gbm.best_iteration)
xgb_baseline = model.predict(xgb.DMatrix(df_test), ntree_limit = model.best_ntree_limit)

In [190]:
lgb_baseline

array([1.45728783, 2.11595454, 1.74336505, ..., 2.45675207, 1.72568508,
       2.33604535])

In [191]:
xgb_baseline

array([1.4491401, 2.1027367, 1.7116534, ..., 2.4396288, 1.6364354,
       2.2246184], dtype=float32)

In [192]:
ensemble = lgb_baseline* 0.25 + xgb_baseline*0.75

In [193]:
ensemble

array([1.45117701, 2.10604123, 1.71958131, ..., 2.44390971, 1.65874778,
       2.25247516])